## Øvelse 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2

seed = 237

# Parameters from Exercise 4
A = 8
m = 10
num_samples = 10000
burn_in = 0

# Defining the unnormalized target distribution
def target_pmf(i):
    return (A ** i) / np.math.factorial(i)

# Metropolis-Hastings algorithm
def metropolis_hastings(A, m, num_samples, burn_in):
    samples = []
    current = np.random.randint(0, m+1)

    for _ in range(num_samples + burn_in):
        # Propose a new state
        if current == 0:
            proposal = current + 1
        elif current == m:
            proposal = current - 1
        else:
            proposal = current + np.random.choice([-1, 1])

        # Computing acceptance ratio
        ratio = target_pmf(proposal) / target_pmf(current)
        alpha = min(1, ratio)

        if np.random.rand() < alpha:
            current = proposal

        samples.append(current)

    return np.array(samples[burn_in:])

# Running the simulation
samples = metropolis_hastings(A, m, num_samples, burn_in)

# Hhistogram of the samples
plt.figure(figsize=(10, 6))
counts, bins, patches = plt.hist(samples, bins=np.arange(m+2)-0.5, density=False, alpha=0.5, edgecolor='grey', color='skyblue')
plt.xticks(np.arange(m+1))
plt.title("Histogram of Metropolis-Hastings samples", fontsize=16)
plt.xlabel("State", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.show()

# Chi test
observed_counts = counts
expected_probs = np.array([target_pmf(i) for i in range(m+1)])
expected_probs /= expected_probs.sum()  # normalizing
expected_counts = expected_probs * num_samples

# Chi-square statistics
chi_square_stat = ((observed_counts - expected_counts) ** 2 / expected_counts).sum()
df = m  # degrees of freedom
p_value = 1 - chi2.cdf(chi_square_stat, df)

print(f"Chi-square statistic: {chi_square_stat:.4f}")
print(f"p-value: {p_value:.4f}")


## Øvelse 2a

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial

# Parameters
A1 = A2 = 4
m = 10
num_samples = 10000
burn_in = 0

# Unnormalized joint distribution
def target_pmf(i, j):
    if i < 0 or j < 0 or i + j > m:
        return 0
    return (A1 ** i) / factorial(i) * (A2 ** j) / factorial(j)

# Proposal: uniform move to one of the 8 neighboring states
def propose(i, j):
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1),
             (-1, -1), (-1, 1), (1, -1), (1, 1)]
    np.random.shuffle(moves)
    for di, dj in moves:
        ni, nj = i + di, j + dj
        if 0 <= ni <= m and 0 <= nj <= m and ni + nj <= m:
            return ni, nj
    return i, j  # if no valid move found

# Metropolis-Hastings sampler
def mh_joint_distribution(A1, A2, m, num_samples, burn_in):
    samples = []
    i, j = 0, 0  # initial state

    for _ in range(num_samples + burn_in):
        i_new, j_new = propose(i, j)
        f_new = target_pmf(i_new, j_new)
        f_old = target_pmf(i, j)

        alpha = min(1, f_new / f_old) if f_old > 0 else 1

        if np.random.rand() < alpha:
            i, j = i_new, j_new

        samples.append((i, j))

    return np.array(samples[burn_in:])

# Run the simulation
samples_2a = mh_joint_distribution(A1, A2, m, num_samples, burn_in)

# Plot a heatmap of frequencies
heatmap = np.zeros((m+1, m+1))
for i, j in samples_2a:
    heatmap[i, j] += 1

plt.figure(figsize=(8, 6))
plt.imshow(heatmap, origin='lower', cmap='Blues', extent=[-0.5, m+0.5, -0.5, m+0.5])
plt.colorbar(label='Frequency')
plt.title('Joint Distribution Samples (Metropolis-Hastings)')
plt.xlabel('i')
plt.ylabel('j')
plt.grid(False)
plt.show()


## Øvelse 2b

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial
from scipy.stats import chi2

# Parameters
A1 = A2 = 4
m = 10
num_samples = 10000
burn_in = 2000

# Unnormalized joint distribution
def target_pmf(i, j):
    if i < 0 or j < 0 or i + j > m:
        return 0
    return (A1 ** i) / factorial(i) * (A2 ** j) / factorial(j)

# Coordinate-wise Metropolis-Hastings sampler
def mh_coordinatewise(A1, A2, m, num_samples, burn_in):
    samples = []
    i, j = 0, 0  # initial state

    for _ in range(num_samples + burn_in):
        # Step 1: Propose change in i
        i_new = i + np.random.choice([-1, 1])
        if 0 <= i_new <= m and i_new + j <= m:
            alpha_i = min(1, target_pmf(i_new, j) / target_pmf(i, j))
            if np.random.rand() < alpha_i:
                i = i_new

        # Step 2: Propose change in j
        j_new = j + np.random.choice([-1, 1])
        if 0 <= j_new <= m and i + j_new <= m:
            alpha_j = min(1, target_pmf(i, j_new) / target_pmf(i, j))
            if np.random.rand() < alpha_j:
                j = j_new

        samples.append((i, j))

    return np.array(samples[burn_in:])

# Compute expected frequencies for chi-squared test
expected = np.zeros((m+1, m+1))
Z = sum(target_pmf(i, j) for i in range(m + 1) for j in range(m + 1 - i))
for i in range(m + 1):
    for j in range(m + 1 - i):
        expected[i, j] = target_pmf(i, j) / Z * num_samples

# Run coordinate-wise MH
samples_2b = mh_coordinatewise(A1, A2, m, num_samples, burn_in)

# Count observed frequencies
heatmap_2b = np.zeros((m+1, m+1))
for i, j in samples_2b:
    heatmap_2b[i, j] += 1

# Compute chi-squared statistic
chi2_stat_2b = 0
df_2b = 0
for i in range(m + 1):
    for j in range(m + 1 - i):
        if expected[i, j] >= 5:
            obs = heatmap_2b[i, j]
            exp = expected[i, j]
            chi2_stat_2b += (obs - exp) ** 2 / exp
            df_2b += 1

p_value_2b = 1 - chi2.cdf(chi2_stat_2b, df_2b - 1)
chi2_stat_2b, p_value_2b

print(f"Chi-squared: {chi2_stat_2b:.4f}")
print(f"P-value: {p_value_2b:.4f}")


In [ ]:
# 2c

# Gibbs sampling implementation
def gibbs_sampler(A1, A2, m, num_samples, burn_in):
    samples = []
    i, j = 0, 0  # Initial state

    for _ in range(num_samples + burn_in):
        # Sample i | j
        max_i = m - j
        weights_i = np.array([(A1**ii / factorial(ii)) for ii in range(max_i + 1)])
        probs_i = weights_i / weights_i.sum()
        i = np.random.choice(np.arange(max_i + 1), p=probs_i)

        # Sample j | i
        max_j = m - i
        weights_j = np.array([(A2**jj / factorial(jj)) for jj in range(max_j + 1)])
        probs_j = weights_j / weights_j.sum()
        j = np.random.choice(np.arange(max_j + 1), p=probs_j)

        samples.append((i, j))

    return np.array(samples[burn_in:])

# Run Gibbs sampler
samples_2c = gibbs_sampler(A1, A2, m, num_samples, burn_in)

# Count observed frequencies
heatmap_2c = np.zeros((m+1, m+1))
for i, j in samples_2c:
    heatmap_2c[i, j] += 1

# Compute chi-squared statistic
chi2_stat_2c = 0
df_2c = 0
for i in range(m + 1):
    for j in range(m + 1 - i):
        if expected[i, j] >= 5:
            obs = heatmap_2c[i, j]
            exp = expected[i, j]
            chi2_stat_2c += (obs - exp) ** 2 / exp
            df_2c += 1

p_value_2c = 1 - chi2.cdf(chi2_stat_2c, df_2c - 1)
chi2_stat_2c, p_value_2c

print(f"Chi-squared (Gibbs): {chi2_stat_2c:.4f}")
print(f"P-value (Gibbs): {p_value_2c:.4f}")

## Øvelse 3

In [ ]:
# a
# Parameters for the bivariate normal
rho = 0.5
mean = [0, 0]
cov = [[1, rho], [rho, 1]]

# Draw one sample from the bivariate normal distribution
xi_gamma = np.random.multivariate_normal(mean, cov)

# Transform to (theta, psi)
theta = np.exp(xi_gamma[0])
psi = np.exp(xi_gamma[1])

theta, psi

print(f"Theta = {theta:.4f}")
print(f"Psi = {psi:.4f}")


In [ ]:
# b

# Number of observations to generate
n = 10

# Generate n observations from N(theta, psi)
observations = np.random.normal(loc=theta, scale=np.sqrt(psi), size=n)

observations


In [ ]:
# c

# Use the same observations from before (as per image, hardcoded here)
observations = np.array([
    0.09117926, 0.95946243, 0.82744338, 1.00187944, -0.42340207,
    -0.45253969, 0.56734008, -0.01968073, 2.40691926, -0.03388189
])

n = len(observations)
x_bar = np.mean(observations)
s2 = np.var(observations, ddof=0)

print(f"Sample mean (x̄) = {x_bar:.4f}")
print(f"Sample variance (s²) = {s2:.4f}")

In [ ]:
# d

# Posterior density function (up to proportionality)
def posterior_density(theta, psi, x, rho=0.5):
    if theta <= 0 or psi <= 0:
        return 0

    n = len(x)
    x_bar = np.mean(x)
    sum_sq = np.sum((x - theta)**2)

    # Log-prior terms
    log_theta = np.log(theta)
    log_psi = np.log(psi)
    exponent_prior = -(log_theta**2 - 2 * rho * log_theta * log_psi + log_psi**2) / (2 * (1 - rho**2))

    # Log-likelihood + log-prior + Jacobian
    log_post = (
        -n / 2 * np.log(psi)
        - sum_sq / (2 * psi)
        - np.log(theta) - np.log(psi)
        + exponent_prior
    )
    return np.exp(log_post)

# Metropolis-Hastings sampling from the posterior
def mh_posterior(x, n_samples=10000, burn_in=2000):
    samples = []
    theta, psi = 1.0, 1.0  # Initial values

    for _ in range(n_samples + burn_in):
        # Propose new values using log-normal random walk
        theta_prop = np.random.lognormal(mean=np.log(theta), sigma=0.2)
        psi_prop = np.random.lognormal(mean=np.log(psi), sigma=0.2)

        # Acceptance ratio
        p_current = posterior_density(theta, psi, x)
        p_proposal = posterior_density(theta_prop, psi_prop, x)
        alpha = min(1, p_proposal / p_current)

        if np.random.rand() < alpha:
            theta, psi = theta_prop, psi_prop

        samples.append((theta, psi))

    return np.array(samples[burn_in:])

# Run the MH algorithm
posterior_samples = mh_posterior(observations)
theta_mean = np.mean(posterior_samples[:, 0])
psi_mean = np.mean(posterior_samples[:, 1])

print(f"Posterior mean of Theta: {theta_mean:.4f}")
print(f"Posterior mean of Psi: {psi_mean:.4f}")


In [ ]:
# e

# Function to rerun MH for different sample sizes (n=100 and n=1000)
def run_experiment(n, theta_true, psi_true):
    # Simulate data from N(theta, psi)
    x = np.random.normal(loc=theta_true, scale=np.sqrt(psi_true), size=n)
    samples = mh_posterior(x)
    theta_mean = np.mean(samples[:, 0])
    psi_mean = np.mean(samples[:, 1])
    return theta_mean, psi_mean

# Reuse theta and psi from 3a
theta_true = 0.5246
psi_true = 0.3998

# Run for n=100 and n=1000
mean_theta_100, mean_psi_100 = run_experiment(100, theta_true, psi_true)
mean_theta_1000, mean_psi_1000 = run_experiment(1000, theta_true, psi_true)

(mean_theta_100, mean_psi_100), (mean_theta_1000, mean_psi_1000)
print(f"Mean Theta (n=100): {mean_theta_100:.4f}")
print(f"Mean Psi (n=100): {mean_psi_100:.4f}")
print(f"Mean Theta (n=1000): {mean_theta_1000:.4f}")
print(f"Mean Psi (n=1000): {mean_psi_1000:.4f}")